# 3. Creating Table 1 for the Therlid Dataset

Table 1 creation process stratified by various variables within the Therlid dataset

## Defines and Imports

In [4]:
# replace with own projectid to pull from bigquery
%env GOOGLE_CLOUD_PROJECT = aiwonglab

env: GOOGLE_CLOUD_PROJECT=aiwonglab


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# needs to be this version of tableone
!pip install tableone==0.8.0
# !pip install tableone

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')

import missingno as msno
import matplotlib.pyplot as plt

import numpy as np
from tableone import TableOne

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Authenticated


In [7]:
%config InlineBackend.figure_format = 'retina'

In [8]:
%%bigquery final_df --project spring-carving-271217

SELECT *
FROM `spring-carving-271217.test.merged_temp_pairs`

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
len(final_df)

13251

In [10]:
final_df["region"].unique()

array(['Midwest', 'West', None, 'South', 'Northeast'], dtype=object)

## Preprocess

In [11]:
vitals =      [c for c in final_df.columns if c.startswith('vitals_')]
cbc =         [c for c in final_df.columns if c.startswith('cbc_')]
coag =        [c for c in final_df.columns if c.startswith('coag_')]
bmp =         [c for c in final_df.columns if c.startswith('bmp_')]
hfp =         [c for c in final_df.columns if c.startswith('hfp_')]
others =      [c for c in final_df.columns if c.startswith('others_')]
sofa_past =   [c for c in final_df.columns if c.startswith('sofa_past_')]
sofa_future = [c for c in final_df.columns if c.startswith('sofa_future_')]
deltas =      [c for c in final_df.columns if c.startswith('delta_')]
abgs =        [c for c in final_df.columns if c.startswith('abg_')]
temp_pairs = ["temp_c_reference", "temp_c_temporal", "pair_type", "total_no_pairs"]

pat_categ = ["hidden_SIRS", "hidden_fever", "hidden_hypothermia", "source_db", "region", "sex_female", "race_ethnicity"]

pat_cont = ["admission_age","weight_admission", "height_admission",
            "BMI_admission","comorbidity_score_value", 'sofa_past_overall_24hr', 'sofa_future_overall_24hr', 'sofa_day1', "los_hosp_dead", "los_hosp_surv",
            "los_ICU_dead", "los_ICU_surv", "total_no_pairs"]

In [12]:
# binary category
final_df['race_ethnicity_white'] = final_df['race_ethnicity'] \
                              .apply(lambda x: "White" if x == "White" \
                                       else "Racial-Ethnic Group")

# continuous variables
final_df['los_hosp_dead'] = final_df[final_df.in_hospital_mortality == 1].los_hospital
final_df['los_hosp_surv'] = final_df[final_df.in_hospital_mortality == 0].los_hospital
final_df['los_ICU_dead'] = final_df[final_df.in_hospital_mortality == 1].los_ICU
final_df['los_ICU_surv'] = final_df[final_df.in_hospital_mortality == 0].los_ICU

In [13]:
x = final_df[final_df["source_db"] == "mimic_iv"]

pd.crosstab(
    x['race_ethnicity'],
    x['sex_female']
)

sex_female,0,1
race_ethnicity,,
American Indian / Alaska Native,5,1
Asian,29,37
Black,75,65
Hispanic OR Latino,32,35
Native Hawaiian / Pacific Islander,11,4
Unknown,177,115
White,582,384


In [14]:
(pd.crosstab(
    x['race_ethnicity'],
    x['sex_female'],
    normalize=True
) * 100).round(2)

sex_female,0,1
race_ethnicity,,
American Indian / Alaska Native,0.32,0.06
Asian,1.87,2.38
Black,4.83,4.19
Hispanic OR Latino,2.06,2.26
Native Hawaiian / Pacific Islander,0.71,0.26
Unknown,11.40,7.41
White,37.50,24.74


## Straitfy by Source DB

### Table One by Patient Level


In [15]:
groupby = ["source_db"]
categ = list(set(pat_categ) - set(groupby))
nonnorm = pat_cont
cols = categ + nonnorm

In [16]:
# double check
data_pat = final_df.groupby(by=["unique_subject_id", "hospital_admission_id"]).first().reset_index()

In [17]:
table1_pat = TableOne(data_pat, columns=cols,
                      groupby=groupby, categorical=categ, nonnormal=nonnorm,
                      missing=True, overall=False, pval=True, htest_name=True)

In [18]:
table1_pat

Grouped by source_db                                                                                                                  
                                                                                         Missing                 eicu                eicu2             mimic_iv P-Value                                       Test
n                                                                                                                4962                 2764                  937                                                   
hidden_fever, n (%)                      False                                                 0          4803 (96.8)          2679 (96.9)           891 (95.1)   0.018                                Chi-squared
                                         True                                                               159 (3.2)             85 (3.1)             46 (4.9)                                                   
region, n (%)                            Midwest                                             324          2658 (57.3)          1357 (49.1)                       <0.001                                Chi-squared
                                         Northeast                                                           11 (0.2)           567 (20.5)          937 (100.0)                                                   
                                         South                                                             586 (12.6)           631 (22.8)                                                                        
                                         West                                                             1384 (29.8)            208 (7.5)                                                                        
hidden_hypothermia, n (%)                False                                                 0          4795 (96.6)          2678 (96.9)           895 (95.5)   0.132                                Chi-squared
                                         True                                                               167 (3.4)             86 (3.1)             42 (4.5)                                                   
sex_female, n (%)                        0                                                     0          2737 (55.2)          1481 (53.6)           525 (56.0)   0.290                                Chi-squared
                                         1                                                                2225 (44.8)          1283 (46.4)           412 (44.0)                                                   
hidden_SIRS, n (%)                       False                                                 0          4636 (93.4)          2593 (93.8)           849 (90.6)   0.002                                Chi-squared
                                         True                                                               326 (6.6)            171 (6.2)             88 (9.4)                                                   
race_ethnicity, n (%)                    American Indian / Alaska Native                       0             34 (0.7)             19 (0.7)              3 (0.3)  <0.001  Chi-squared (warning: expected count < 5)
                                         Asian                                                               45 (0.9)             71 (2.6)             34 (3.6)                                                   
                                         Black                                                              310 (6.2)           433 (15.7)             84 (9.0)                                                   
                                         Hispanic OR Latino                                                 149 (3.0)             50 (1.8)             41 (4.4)                                                   
                                         Unknown                   

###  Table One by Pairs Level

In [19]:
groupby = ["source_db"]
categ = ["hidden_SIRS", "hidden_fever", "hidden_hypothermia"]
nonnorm =  abgs + vitals + cbc + coag + bmp + hfp + others + sofa_past + sofa_future
cols = categ + nonnorm

In [20]:
table1_pair = TableOne(final_df, columns=cols,
                       groupby=groupby, categorical=categ, nonnormal=nonnorm,
                       missing=True, overall=False)

In [21]:
with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pair)

Grouped by source_db                                                               
                                                                   Missing                 eicu                eicu2             mimic_iv
n                                                                                          7307                 4392                 1552
hidden_SIRS, n (%)                              False                    0          6823 (93.4)          4143 (94.3)          1414 (91.1)
                                                True                                  484 (6.6)            249 (5.7)            138 (8.9)
hidden_fever, n (%)                             False                    0          7066 (96.7)          4276 (97.4)          1481 (95.4)
                                                True                                  241 (3.3)            116 (2.6)             71 (4.6)
hidden_hypothermia, n (%)                       False                    0          7064 (96.7)          4259 (97.0)          1485 (95.7)
                                                True                                  243 (3.3)            133 (3.0)             67 (4.3)
abg_Carboxyhemoglobin, median [Q1,Q3]                                12986        1.0 [0.6,1.6]        0.6 [0.3,1.1]        1.0 [1.0,1.8]
abg_Methemoglobin, median [Q1,Q3]                                    12971        0.4 [0.2,0.7]        0.3 [0.3,0.6]        0.0 [0.0,0.2]
abg_SaO2, median [Q1,Q3]                                             11581     97.2 [95.0,99.0]     98.0 [95.0,99.1]     95.0 [80.8,97.0]
abg_pH, median [Q1,Q3]                                               10236        7.4 [7.3,7.4]        7.4 [7.3,7.4]        7.4 [7.3,7.4]
abg_paCO2, median [Q1,Q3]                                            10165     39.0 [34.0,45.0]     38.0 [34.0,45.0]     39.0 [34.0,44.0]
abg_paO2, median [Q1,Q3]                                             10330   120.0 [80.0,212.2]   130.0 [82.0,191.8]   112.0 [87.0,156.0]
vitals_heart_rate_min, median [Q1,Q3]                                  754     84.0 [72.0,97.0]     84.0 [72.0,96.0]    87.0 [74.0,105.0]
vitals_resp_rate, median [Q1,Q3]                                       150     18.0 [16.0,20.0]     18.0 [16.0,20.0]     20.0 [16.0,25.0]
vitals_mbp_ni, median [Q1,Q3]                                         2724     85.0 [74.0,96.0]     89.0 [79.0,99.0]     75.0 [66.0,87.0]
vitals_sbp_ni, median [Q1,Q3]                                         1238  125.0 [110.0,142.0]  125.0 [111.0,141.0]   113.5 [99.0,128.0]
vitals_dbp_ni, median [Q1,Q3]                                         1238     68.0 [59.0,78.0]     68.0 [60.0,78.0]     62.0 [53.0,73.0]
vitals_mbp_i, median [Q1,Q3]                                          8739     85.0 [74.0,96.0]     84.0 [74.0,94.0]     74.0 [66.0,84.0]
vitals_sbp_i, median [Q1,Q3]                                          9038  128.0 [111.0,145.8]  128.0 [112.0,143.0]  112.0 [100.0,127.0]
vitals_dbp_i, median [Q1,Q3]                                          9040     61.0 [53.0,70.0]     62.0 [53.0,71.0]     56.0 [49.0,64.0]
vitals_tempc_0_2h_before, median [Q1,Q3]                              5690     36.7 [36.4,37.2]     36.6 [36.3,37.0]     36.8 [36.2,37.4]
vitals_tempc_2h_4h_before, median [Q1,Q3]                             6462     36.8 [36.4,37.2]     36.7 [36.4,37.0]     36.9 [36.4,37.4]
vitals_tempc_4h_8h_before, median [Q1,Q3]                             4556     36.8 [36.5,37.1]     36.7 [36.5,37.0]     36.9 [36.4,37.3]
vitals_tempc_8h_12h_before, median [Q1,Q3]                            5419     36.8 [36.6,37.1]     36.7 [36.5,37.0]     36.9 [36.5,37.4]
vitals_tempc_12h_20h_before, median [Q1,Q3]                           5094     36.8 [36.6,37.2]     36.8 [36.5,37.0]     36.9 [36.6,37.3]
vitals_tempc_20h_28h_before, median [Q1,Q3]                           6038     36.8 [36.5,37.1]     36.8 [36.5,37.0]     36.9 [36.6,37.3]
vitals_tempc_28h_36h_before, median [Q1,Q3]         

## Stratify by Race and Ethnicity

### Table One by Patient Level

In [22]:
groupby = ["race_ethnicity"]
categ = list(set(pat_categ) - set(groupby))
nonnorm = pat_cont
cols = categ + nonnorm

data_pat = final_df.groupby(by=["unique_subject_id", "hospital_admission_id"]).first().reset_index()

table1_pat = TableOne(data_pat, columns=cols,
                      groupby=groupby, categorical=categ, nonnormal=nonnorm,
                      missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pat)

Grouped by race_ethnicity                                                                                                                                                                            
                                                                     Missing American Indian / Alaska Native                Asian                Black   Hispanic OR Latino Native Hawaiian / Pacific Islander              Unknown                White
n                                                                                                         56                  150                  827                  240                                  6                  590                 6794
source_db, n (%)                         eicu                              0                       34 (60.7)            45 (30.0)           310 (37.5)           149 (62.1)                                              212 (35.9)          4212 (62.0)
                                         eicu2                                                     19 (33.9)            71 (47.3)           433 (52.4)            50 (20.8)                                              187 (31.7)          2004 (29.5)
                                         mimic_iv                                                    3 (5.4)            34 (22.7)            84 (10.2)            41 (17.1)                          6 (100.0)           191 (32.4)            578 (8.5)
hidden_fever, n (%)                      False                             0                       52 (92.9)           141 (94.0)           790 (95.5)           227 (94.6)                          6 (100.0)           554 (93.9)          6603 (97.2)
                                         True                                                        4 (7.1)              9 (6.0)             37 (4.5)             13 (5.4)                                                36 (6.1)            191 (2.8)
region, n (%)                            Midwest                         324                       21 (37.5)            40 (27.2)           294 (36.1)            41 (17.9)                                               95 (16.4)          3524 (54.2)
                                         Northeast                                                  7 (12.5)            70 (47.6)           157 (19.3)            82 (35.8)                          6 (100.0)           290 (50.1)           903 (13.9)
                                         South                                                     12 (21.4)            21 (14.3)           326 (40.0)             14 (6.1)                                               71 (12.3)           773 (11.9)
                                         West                                                      16 (28.6)            16 (10.9)             38 (4.7)            92 (40.2)                                              123 (21.2)          1307 (20.1)
sex_female, n (%)                        0                                 0                       32 (57.1)            82 (54.7)           415 (50.2)           123 (51.2)                           3 (50.0)           340 (57.6)          3748 (55.2)
                                         1                                                         24 (42.9)            68 (45.3)           412 (49.8)           117 (48.8)                           3 (50.0)           250 (42.4)          3046 (44.8)
hidden_hypothermia, n (%)                False                             0                       54 (96.4)           143 (95.3)           786 (95.0)           232 (96.7)                          6 (100.0)           569 (96.4)          6578 (96.8)
                                         True                                                        2 (3.6)              7 (4.7)             41 (5.0)              8 (3.3)                                                21 (3.6)            216 (3.2)
hidden_SIRS, n (%)                       False                     

### Table One by Pair Level

In [23]:
groupby = ["race_ethnicity_white"]
categ = ["hidden_SIRS", "hidden_fever", "hidden_hypothermia"]
nonnorm =  abgs + vitals + cbc + coag + bmp + hfp + others + sofa_past + sofa_future
cols = categ + nonnorm

table1_pair = TableOne(final_df, columns=cols,
                       groupby=groupby, categorical=categ, nonnormal=nonnorm,
                       missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pair)

Grouped by race_ethnicity_white                                          
                                                                              Missing  Racial-Ethnic Group                White
n                                                                                                     2894                10357
hidden_SIRS, n (%)                              False                               0          2634 (91.0)          9746 (94.1)
                                                True                                             260 (9.0)            611 (5.9)
hidden_fever, n (%)                             False                               0          2752 (95.1)         10071 (97.2)
                                                True                                             142 (4.9)            286 (2.8)
hidden_hypothermia, n (%)                       False                               0          2776 (95.9)         10032 (96.9)
                                                True                                             118 (4.1)            325 (3.1)
abg_Carboxyhemoglobin, median [Q1,Q3]                                           12986        0.8 [0.3,1.1]        1.0 [0.4,1.6]
abg_Methemoglobin, median [Q1,Q3]                                               12971        0.3 [0.2,0.4]        0.4 [0.2,0.9]
abg_SaO2, median [Q1,Q3]                                                        11581     97.0 [93.1,99.0]     97.0 [94.5,99.0]
abg_pH, median [Q1,Q3]                                                          10236        7.4 [7.3,7.4]        7.4 [7.3,7.4]
abg_paCO2, median [Q1,Q3]                                                       10165     38.0 [34.0,44.0]     39.0 [34.0,45.0]
abg_paO2, median [Q1,Q3]                                                        10330   117.0 [86.0,177.0]   122.0 [82.8,195.0]
vitals_heart_rate_min, median [Q1,Q3]                                             754    86.0 [73.0,101.0]     84.0 [72.0,97.0]
vitals_resp_rate, median [Q1,Q3]                                                  150     18.0 [16.0,22.0]     18.0 [16.0,21.0]
vitals_mbp_ni, median [Q1,Q3]                                                    2724     87.0 [76.0,99.0]     86.0 [75.0,96.0]
vitals_sbp_ni, median [Q1,Q3]                                                    1238  125.0 [109.0,143.0]  124.0 [110.0,140.0]
vitals_dbp_ni, median [Q1,Q3]                                                    1238     69.0 [60.0,79.0]     67.0 [58.0,77.0]
vitals_mbp_i, median [Q1,Q3]                                                     8739     82.0 [71.0,93.0]     82.0 [72.0,93.0]
vitals_sbp_i, median [Q1,Q3]                                                     9038  121.0 [106.0,140.0]  125.0 [109.0,142.0]
vitals_dbp_i, median [Q1,Q3]                                                     9040     61.0 [51.0,70.0]     60.0 [52.0,69.0]
vitals_tempc_0_2h_before, median [Q1,Q3]                                         5690     36.7 [36.3,37.2]     36.7 [36.3,37.1]
vitals_tempc_2h_4h_before, median [Q1,Q3]                                        6462     36.8 [36.4,37.2]     36.8 [36.4,37.2]
vitals_tempc_4h_8h_before, median [Q1,Q3]                                        4556     36.8 [36.5,37.2]     36.8 [36.5,37.1]
vitals_tempc_8h_12h_before, median [Q1,Q3]                                       5419     36.8 [36.6,37.2]     36.8 [36.5,37.1]
vitals_tempc_12h_20h_before, median [Q1,Q3]                                      5094     36.8 [36.6,37.1]     36.8 [36.6,37.1]
vitals_tempc_20h_28h_before, median [Q1,Q3]                                      6038     36.8 [36.5,37.1]     36.8 [36.5,37.1]
vitals_tempc_28h_36h_before, median [Q1,Q3]                                      6944     36.8 [36.6,37.1]     36.8 [36.5,37.1]
vitals_tempc_36h_48h_before, median [Q1,Q3]                                      7123     36.8 [36.6,37.2]     36.8 [36.6,37.1]
vitals_spo2, median [Q1,Q3]                                                      1257 

## Stratify by Hidden SIRS

### Table One by Patient Level

In [24]:
groupby = ["hidden_SIRS"]
categ = list(set(pat_categ) - set(groupby))
nonnorm = pat_cont
cols = categ + nonnorm

data_pat = final_df.groupby(by=["unique_subject_id", "hospital_admission_id"]).first().reset_index()

table1_pat = TableOne(data_pat, columns=cols,
                      groupby=groupby, categorical=categ, nonnormal=nonnorm,
                      missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pat)

Grouped by hidden_SIRS                                          
                                                                                           Missing                False                 True
n                                                                                                                  8078                  585
source_db, n (%)                         eicu                                                    0          4636 (57.4)           326 (55.7)
                                         eicu2                                                              2593 (32.1)           171 (29.2)
                                         mimic_iv                                                            849 (10.5)            88 (15.0)
hidden_fever, n (%)                      False                                                   0         8078 (100.0)           295 (50.4)
                                         True                                                                                     290 (49.6)
region, n (%)                            Midwest                                               324          3799 (48.9)           216 (38.1)
                                         Northeast                                                          1358 (17.5)           157 (27.7)
                                         South                                                              1122 (14.4)            95 (16.8)
                                         West                                                               1493 (19.2)            99 (17.5)
sex_female, n (%)                        0                                                       0          4404 (54.5)           339 (57.9)
                                         1                                                                  3674 (45.5)           246 (42.1)
hidden_hypothermia, n (%)                False                                                   0         8078 (100.0)           290 (49.6)
                                         True                                                                                     295 (50.4)
race_ethnicity, n (%)                    American Indian / Alaska Native                         0             50 (0.6)              6 (1.0)
                                         Asian                                                                134 (1.7)             16 (2.7)
                                         Black                                                                749 (9.3)            78 (13.3)
                                         Hispanic OR Latino                                                   219 (2.7)             21 (3.6)
                                         Native Hawaiian / Pacific Islander                                     6 (0.1)                     
                                         Unknown                                                              533 (6.6)             57 (9.7)
                                         White                                                              6387 (79.1)           407 (69.6)
admission_age, median [Q1,Q3]                                                                    0     64.0 [52.0,74.0]     65.0 [53.0,75.0]
weight_admission, median [Q1,Q3]                                                               296     81.0 [67.5,97.5]     80.5 [66.4,97.1]
height_admission, median [Q1,Q3]                                                               374  170.2 [162.6,177.8]  170.2 [162.6,177.8]
BMI_admission, median [Q1,Q3]                                                                  553     27.8 [23.9,32.9]     27.8 [23.6,32.5]
comorbidity_score_value, median [Q1,Q3]                                                          0        3.0 [2.0,5.0]        4.0 [2.0,5.0]
sofa_past_overall_24hr, median [Q1,Q3]                                                        5276        4.0 [2.0,7.0]        4

### Table One by Pair Level

In [25]:
groupby = ["hidden_SIRS"]
categ = ["hidden_fever", "hidden_hypothermia"]
nonnorm =  abgs + vitals + cbc + coag + bmp + hfp + others + sofa_past + sofa_future
cols = categ + nonnorm

table1_pair = TableOne(final_df, columns=cols,
                       groupby=groupby, categorical=categ, nonnormal=nonnorm,
                       missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pair)

Grouped by hidden_SIRS                                          
                                                                     Missing                False                 True
n                                                                                           12380                  871
hidden_fever, n (%)                             False                      0        12380 (100.0)           443 (50.9)
                                                True                                                        428 (49.1)
hidden_hypothermia, n (%)                       False                      0        12380 (100.0)           428 (49.1)
                                                True                                                        443 (50.9)
abg_Carboxyhemoglobin, median [Q1,Q3]                                  12986        0.9 [0.3,1.4]        0.6 [0.3,1.0]
abg_Methemoglobin, median [Q1,Q3]                                      12971        0.3 [0.2,0.7]        0.3 [0.2,0.5]
abg_SaO2, median [Q1,Q3]                                               11581     97.0 [94.5,99.0]     96.3 [92.4,98.0]
abg_pH, median [Q1,Q3]                                                 10236        7.4 [7.3,7.4]        7.4 [7.3,7.4]
abg_paCO2, median [Q1,Q3]                                              10165     39.0 [34.0,44.0]     39.0 [32.0,46.0]
abg_paO2, median [Q1,Q3]                                               10330   123.0 [84.0,194.0]    99.0 [78.0,149.0]
vitals_heart_rate_min, median [Q1,Q3]                                    754     84.0 [72.0,98.0]    86.0 [74.0,104.0]
vitals_resp_rate, median [Q1,Q3]                                         150     18.0 [16.0,21.0]     19.0 [16.0,23.0]
vitals_mbp_ni, median [Q1,Q3]                                           2724     86.0 [76.0,97.0]     83.0 [72.0,96.0]
vitals_sbp_ni, median [Q1,Q3]                                           1238  124.0 [110.0,141.0]  121.0 [104.0,140.0]
vitals_dbp_ni, median [Q1,Q3]                                           1238     68.0 [59.0,78.0]     65.0 [56.0,76.0]
vitals_mbp_i, median [Q1,Q3]                                            8739     82.0 [72.0,93.0]     80.0 [69.0,92.0]
vitals_sbp_i, median [Q1,Q3]                                            9038  125.0 [109.0,142.0]  117.0 [102.8,142.2]
vitals_dbp_i, median [Q1,Q3]                                            9040     60.0 [52.0,69.0]     58.0 [50.0,66.0]
vitals_tempc_0_2h_before, median [Q1,Q3]                                5690     36.7 [36.3,37.1]     36.7 [36.1,37.9]
vitals_tempc_2h_4h_before, median [Q1,Q3]                               6462     36.8 [36.4,37.1]     36.8 [36.3,37.6]
vitals_tempc_4h_8h_before, median [Q1,Q3]                               4556     36.8 [36.5,37.1]     36.9 [36.4,37.5]
vitals_tempc_8h_12h_before, median [Q1,Q3]                              5419     36.8 [36.6,37.1]     36.8 [36.4,37.4]
vitals_tempc_12h_20h_before, median [Q1,Q3]                             5094     36.8 [36.6,37.1]     36.8 [36.3,37.4]
vitals_tempc_20h_28h_before, median [Q1,Q3]                             6038     36.8 [36.5,37.1]     36.8 [36.4,37.4]
vitals_tempc_28h_36h_before, median [Q1,Q3]                             6944     36.8 [36.6,37.1]     36.8 [36.4,37.4]
vitals_tempc_36h_48h_before, median [Q1,Q3]                             7123     36.8 [36.6,37.1]     36.9 [36.4,37.4]
vitals_spo2, median [Q1,Q3]                                             1257     97.0 [95.0,99.0]     97.0 [95.0,99.0]
cbc_hemoglobin, median [Q1,Q3]                                          2608       9.9 [8.3,12.0]      10.0 [8.5,12.0]
cbc_hematocrit, median [Q1,Q3]                                          3014     30.2 [25.2,36.0]     31.0 [26.4,37.0]
cbc_mch, median [Q1,Q3]                                                 5072     29.9 [28.4,31.2]     29.9 [28.1,31.0]
cbc_mchc, median [Q1,Q3]                                                3396     33.0 [32.0,34.0]     32.7 [31.6,33.7]
cbc_mcv,